In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*5*5, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Instantiate and train MLP
print("Training MLP...")
mlp_model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)
train_model(mlp_model, train_loader, criterion, optimizer, num_epochs=10)
print("Testing MLP...")
mlp_accuracy = test_model(mlp_model, test_loader)

# Instantiate and train CNN
print("Training CNN...")
cnn_model = CNN().to(device)
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
train_model(cnn_model, train_loader, criterion, optimizer, num_epochs=10)
print("Testing CNN...")
cnn_accuracy = test_model(cnn_model, test_loader)

# Compare performances
print("Comparison:")
print(f"MLP Test Accuracy: {mlp_accuracy:.2f}%")
print(f"CNN Test Accuracy: {cnn_accuracy:.2f}%")


Training MLP...
Epoch [1/10], Loss: 0.3978
Epoch [2/10], Loss: 0.2167
Epoch [3/10], Loss: 0.1776
Epoch [4/10], Loss: 0.1597
Epoch [5/10], Loss: 0.1463
Epoch [6/10], Loss: 0.1381
Epoch [7/10], Loss: 0.1262
Epoch [8/10], Loss: 0.1214
Epoch [9/10], Loss: 0.1182
Epoch [10/10], Loss: 0.1110
Testing MLP...
Test Accuracy: 97.25%
Training CNN...
Epoch [1/10], Loss: 0.2340
Epoch [2/10], Loss: 0.0892
Epoch [3/10], Loss: 0.0677
Epoch [4/10], Loss: 0.0543
Epoch [5/10], Loss: 0.0470
Epoch [6/10], Loss: 0.0383
Epoch [7/10], Loss: 0.0360
Epoch [8/10], Loss: 0.0312
Epoch [9/10], Loss: 0.0275
Epoch [10/10], Loss: 0.0265
Testing CNN...
Test Accuracy: 99.14%
Comparison:
MLP Test Accuracy: 97.25%
CNN Test Accuracy: 99.14%
